In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

In [10]:
from typing import TypedDict, List, Dict, Any
from datetime import datetime
from langgraph.graph import StateGraph
import operator
import json
import ast

class AgentState(TypedDict):
    task: str
    code: str
    status: str
    error: str | None
    context: Dict[str, Any]
    
def validate_python(code: str) -> bool:
    """Validate Python code syntax"""
    try:
        ast.parse(code)
        return True
    except Exception:
        return False

def analyze_task(state: dict) -> dict:
    """Analyze the task and generate initial code"""
    task = state["task"]
    
    # Example code generation based on task
    # In real implementation, this would use an LLM
    example_code = {
        "sort numbers": """
def sort_numbers(numbers):
    if not isinstance(numbers, list):
        raise ValueError("Input must be a list")
    return sorted(numbers)
        """,
        "process csv": """
import pandas as pd

def process_csv(filepath):
    df = pd.read_csv(filepath)
    return df.describe()
        """
    }
    
    state["code"] = example_code.get(
        task.lower(),
        f"# TODO: Implement {task}"
    )
    state["status"] = "generated"
    
    return state

def validate_code(state: dict) -> dict:
    """Validate the generated code"""
    code = state["code"]
    
    if validate_python(code):
        state["status"] = "validated"
    else:
        state["status"] = "error"
        state["error"] = "Invalid Python syntax"
    
    return state

def execute_code(state: dict) -> dict:
    """Execute the validated code"""
    if state["status"] != "validated":
        return state
        
    try:
        # Create execution environment
        exec_globals = {}
        exec(state["code"], exec_globals)
        
        # Try to execute with context if available
        if "test_data" in state["context"]:
            result = exec_globals["sort_numbers"](state["context"]["test_data"])
            state["context"]["result"] = result
            
        state["status"] = "completed"
        
    except Exception as e:
        state["status"] = "error"
        state["error"] = str(e)
    
    return state

def create_agent() -> StateGraph:
    """Create the agent workflow"""
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("analyze", analyze_task)
    workflow.add_node("validate", validate_code)
    workflow.add_node("execute", execute_code)
    
    # Add edges
    workflow.add_edge("analyze", "validate")
    workflow.add_edge("validate", "execute")
    
    # Set entry point
    workflow.set_entry_point("analyze")
    
    return workflow.compile()

def run_agent(task: str, context: Dict = None) -> Dict:
    """Run the code generation agent"""
    # Create agent
    agent = create_agent()
    
    # Initial state
    initial_state = {
        "task": task,
        "code": "",
        "status": "starting",
        "error": None,
        "context": context or {}
    }
    
    # Run agent
    final_state = agent.invoke(initial_state)
    
    # Format result
    return {
        "task": final_state["task"],
        "code": final_state["code"],
        "status": final_state["status"],
        "error": final_state["error"],
        "result": final_state["context"].get("result"),
    }

if __name__ == "__main__":
    # Test: Sort numbers
    result = run_agent("sort numbers", {
        "test_data": [5, 2, 8, 1, 9]
    })
    print(json.dumps(result, indent=2))

{
  "task": "sort numbers",
  "code": "\ndef sort_numbers(numbers):\n    if not isinstance(numbers, list):\n        raise ValueError(\"Input must be a list\")\n    return sorted(numbers)\n        ",
  "status": "completed",
  "error": null,
  "result": [
    1,
    2,
    5,
    8,
    9
  ]
}
